## Travel Agent, Online MCP

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage

# 1️⃣ Connect to the Kiwi MCP server
client = MultiServerMCPClient({
    "travel_server": {
        "transport": "streamable_http",
        "url": "https://mcp.kiwi.com"
    }
})

tools = await client.get_tools()  # fetch tools
print(tools)  # should show search-flight

# 2️⃣ Create the agent with the fetched tools
agent = create_agent(
    "gpt-5-nano",
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt="You are a travel agent. No follow up questions."
)

config = {"configurable": {"thread_id": "1"}}

# 3️⃣ Ask for a flight — use async invocation
response = await agent.ainvoke(
    {"messages": [HumanMessage(
        content="Get me a direct flight from San Francisco to New York on March 1st 2026",
    )]},
    config=config
)

print(response)


[StructuredTool(name='search-flight', description='\n# Search for a flight\n\n## Description\n\nUses the Kiwi API to search for available flights between two locations on a specific date.\n\n## How it works\n\nThe tool will:\n1. Search for matching locations to resolve airport codes\n2. Find available flights for the specified route and date range\n\n## Method\n\nCall this tool whenever a user wants to search for flights, regardless of whether they provided exact airport codes or just city names.\n\nYou should display the returned results in a markdown table format: Group the results by price (those who are the cheapest), duration (those who are the shortest, i.e. have the smallest \'totalDurationInSeconds\') and the rest (those that could still be interesting).\n\nAlways display for each flight in order:\n  - In the 1st column: The departure and arrival airports, including layovers (e.g. "Paris CDG → Barcelona BCN → Lisbon LIS")\n  - In the 2nd column: The departure and arrival dates 

In [3]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Get me a direct flight from San Francisco to New York on March 1st 2026', additional_kwargs={}, response_metadata={}, id='eb936141-29e1-4bf3-80a9-e9b79d319e29'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 844, 'prompt_tokens': 1233, 'total_tokens': 2077, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CpQ5zMO7wZB5hvvDeOXn2yj4rriwN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b43eb-a089-7ef2-bae9-168871da4001-0', tool_calls=[{'name': 'search-flight', 'args': {'flyFrom': 'San Francisco', 'flyTo': 'New York', 'departureDate': '01/03/2026', 'passengers': {

In [4]:
print(response["messages"][-1].content)


Here are direct (non-stop) San Francisco (SFO) to New York (NYC) options for March 1, 2026. Prices are shown in USD.

Cheapest direct options (107 USD)
- Route: SFO → JFK
  - When: 01/03/2026 06:29 (local) to 01/03/2026 15:00 (local) – Duration: 5h 31m
  - Cabin: Economy
  - Return: N/A
  - Return times: N/A
  - Return cabin: N/A
  - Price: 107 USD
  - Book: https://on.kiwi.com/8hU9SH

- Route: SFO → JFK
  - When: 01/03/2026 22:54 (local) to 02/03/2026 07:30 (local) – Duration: 5h 31m
  - Cabin: Economy
  - Return: N/A
  - Return times: N/A
  - Return cabin: N/A
  - Price: 107 USD
  - Book: https://on.kiwi.com/fh5Div

Shortest-duration direct options (by total flight duration)
- Route: SFO → EWR
  - When: 01/03/2026 08:00 (local) to 01/03/2026 16:30 (local) – Duration: 5h 30m
  - Cabin: Economy
  - Return: N/A
  - Return times: N/A
  - Return cabin: N/A
  - Price: 110 USD
  - Book: https://on.kiwi.com/AKgV2m

- Route: SFO → JFK
  - When: 01/03/2026 17:00 (local) to 01/03/2026 22:36 (lo